<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Deeplearning_Budget_and_Forecasting_LSTMv1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx to Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
tf.config.run_functions_eagerly(True)
# Read the data
# file_path = 'E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx'
# df = pd.read_excel(file_path)
file_path = 'Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx'
df = pd.read_excel(file_path)
# Drop rows with missing values in the specified columns
df = df.dropna(subset=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])

# Convert the values in columns "01" to "12" to numeric
df[["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]] = df[
    ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
].apply(pd.to_numeric, errors="coerce")

# Define months and years
months = [f"{i:02d}" for i in range(1, 13)]
years = df["Year"].unique()

# Group the data based on categories
grouped_df = df.groupby(["MRL_LINE"])

# Filter data for each year (2019, 2020, 2021) and apply ARIMA model for each MRL_LINE
years = [2019, 2020, 2021, 2022]

# Helper function to find the last financial year
def find_last_financial_year(years):
    sorted_years = sorted(years, reverse=True)
    for i in range(len(sorted_years) - 1):
        if sorted_years[i] - 1 != sorted_years[i + 1]:
            return sorted_years[i]
    return sorted_years[-1]

# Initialize dataframe to store forecast results
predicted_df = pd.DataFrame(columns=["MRL_LINE", "Year", "Month", "LBGM_Predicted_Value"])
forecast_df = pd.DataFrame(columns=["MRL_LINE", "Year", "Month", "LBGM_Forecasted_Value"])
metrics_df = pd.DataFrame(
    columns=["MRL", "MSE", "RMSE", "MAPE", "z_scores_t", "z_scores_p", "R2"]
)

# Initialize lists to store evaluation metrics
mse_list = []
r2_list = []
rmse_list = []
mape_list = []
z_scores_list_t = []
z_scores_list_p = []

# Define the create_sequences function
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

for mrl_line, group_df in grouped_df:
    for year in years:
        year_df = group_df[group_df['Year'] == year]

        # Check if all 12 columns exist in the DataFrame
        data_columns = [str(month).zfill(2) for month in range(1, 13)]
        missing_columns = [col for col in data_columns if col not in year_df.columns]

        if missing_columns:
            print(f"Missing data columns for MRL_LINE: {mrl_line}, Year: {year}: {missing_columns}")
            continue

        data = year_df[data_columns].values.flatten()

        # Handle cases where there might be missing data for some years
        if np.any((data != 0) & pd.notnull(data)):
            # Find last financial year
            last_financial_year = find_last_financial_year(years)

            # Split data into train and test sets
            if year == last_financial_year:
                # Last 12 months as test data
                train_data = data[:-12]
                test_data = data[-12:]
            else:
                # Rest as train data
                train_data = data[:-12]  # Use all available data except the last 12 months
                test_data = data[-12:]  # Use the last 12 months as test data
        # Normalize the data
        scaler = MinMaxScaler()
        train_data = scaler.fit_transform(train_data.reshape(-1, 1))
        test_data = scaler.transform(test_data.reshape(-1, 1))

        # # Create sequences of data
        # n_steps = 12  # Number of previous months to consider for prediction
        # X_train, y_train = create_sequences(train_data, n_steps)
        # X_test, y_test = create_sequences(test_data, n_steps)


        # Create sequences of data
        n_steps_train = 24  # Number of previous months to consider for prediction in training data
        n_steps_test = 12   # Number of previous months to consider for prediction in test data
        X_train, y_train = create_sequences(train_data, n_steps_train)
        X_test, y_test = create_sequences(test_data, n_steps_test)

        # Reshape the input data
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        # X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        X_test = np.reshape(X_test, (X_test.shape[0], n_steps_test, 1))
        # # Reshape the input data
        # X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        # X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        # Build the Bidirectional LSTM model
        model = Sequential()
        model.add(
            Bidirectional(LSTM(units=128, activation="relu"), input_shape=(X_train.shape[1], X_train.shape[2]))
        )
        model.add(Dense(1))  # Output layer

        # Compile the model
        # model.compile(optimizer="adam", loss="mean_squared_error")
        model.compile(optimizer="adam", loss="mean_squared_error")


        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

        # Train the model
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)

        # Make predictions
        predictions_scaled = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions_scaled)

        # Append predicted values to the dataframe
        predicted_values = predictions.flatten()
        months_range = pd.date_range(start=f"{year}-01-01", periods=12, freq="M")
        predicted_df = predicted_df.append(
            pd.DataFrame(
                {
                    "MRL_LINE": mrl_line,
                    "Year": year,
                    "Month": months_range,
                    "LSTM_Predicted_Value": predicted_values,
                }
            )
        )

        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, predictions)


        mse_list.append(mse)
        rmse_list.append(rmse)
        r2_list.append(r2)

# The key change here is that we are using y_test instead of test_data when calculating the evaluation metrics. Also, y_test is used directly for calculating the mean squared error and other metrics. This should resolve the IndexError issue.

# Please replace the above code snippet in the appropriate place in the full code provided earlier.

        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, predictions)

        mse_list.append(mse)
        rmse_list.append(rmse)
        r2_list.append(r2)

        # Combine evaluation metrics into metrics dataframe
        metrics_df["MSE"] = mse_list
        metrics_df["RMSE"] = rmse_list
        metrics_df["R2"] = r2_list

        # Print metrics dataframe
        print(metrics_df)

# Export forecast results to Excel

# Export forecast results to Excel
predicted_output_file = 'E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\DS\\DL\\Sunoida\\LSTM\\predicted_results.xlsx'
predicted_df.to_excel(predicted_output_file, index=False)

print("Forecasting and evaluation completed.")

<ipython-input-5-b71b4c968282>:65: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for mrl_line, group_df in grouped_df:
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: ignored